## 데이터 불러오기

In [31]:
import os
import pandas as pd
PATH = './data/서울시 상권분석서비스(오탈자 수정)/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['rent','small_rent','big_rent','living_popul','indicator','resident_popul', 'income','apartment', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

## Data Preprocessing

### 상주인구(resident_popul) 테이블

#### 불필요한 feature 제거
- 총 상주인구 수
- 남성 상주인구 수
- 여성 상주인구 수
- 연령대 10 ~ 60 이상상주인구 수
- 총 가구 수, 아파트 가구 수, 비아파트 가구수

In [13]:
resident_popul.drop(['총_상주인구_수','남성_상주인구_수','여성_상주인구_수','연령대_10_상주인구_수','연령대_20_상주인구_수','연령대_30_상주인구_수','연령대_40_상주인구_수','연령대_50_상주인구_수','연령대_60_이상_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수'],axis=1,inplace=True)

In [15]:
resident_popul.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
0,20194,D,발달상권,3120021,시청역_8번,0,2,1,1,2,10,0,0,2,0,2,6
1,20194,D,발달상권,3120022,북창동(시청역_6번),0,11,31,28,27,70,2,10,9,7,23,43
2,20194,D,발달상권,3120023,서울시청,0,0,0,2,2,6,2,0,0,1,3,3
3,20194,D,발달상권,3120024,회현역,44,64,97,123,150,424,56,72,74,91,127,418
4,20194,D,발달상권,3120025,롯데백화점(시청광장 지하쇼핑센터),0,0,0,0,0,1,0,0,0,0,0,0


#### 결측치 확인 및 제거
- 결측값 없음

In [14]:
resident_popul.isnull().sum()

기준_년분기_코드             0
상권_구분_코드              0
상권_구분_코드_명            0
상권_코드                 0
상권_코드_명               0
남성연령대_10_상주인구_수       0
남성연령대_20_상주인구_수       0
남성연령대_30_상주인구_수       0
남성연령대_40_상주인구_수       0
남성연령대_50_상주인구_수       0
남성연령대_60_이상_상주인구_수    0
여성연령대_10_상주인구_수       0
여성연령대_20_상주인구_수       0
여성연령대_30_상주인구_수       0
여성연령대_40_상주인구_수       0
여성연령대_50_상주인구_수       0
여성연령대_60_이상_상주인구_수    0
dtype: int64

#### 이상치 확인 및 제거
- 이상치가 없다고 판단됨

In [17]:
resident_popul.describe()

,기준_년분기_코드,상권_코드,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
count,24486.000000,2.448600e+04,24486.000000,24486.000000,24486.000000,24486.000000,24486.000000,24486.000000,24486.000000,24486.000000,24486.00000,24486.000000,24486.000000,24486.000000
mean,20213.785347,3.115106e+06,128.947480,187.723761,206.922323,182.484318,186.750756,273.117782,122.422445,204.107327,194.90321,173.011435,185.558932,324.451973
std,11.113964,1.029469e+04,145.620462,207.389748,226.065609,186.616366,184.753590,269.620920,138.540790,239.474579,216.62858,181.468400,190.638033,323.217719
min,20194.000000,3.001491e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,20203.000000,3.110403e+06,29.000000,45.000000,51.000000,49.000000,51.000000,76.000000,28.000000,46.000000,47.00000,44.000000,46.000000,87.000000
50%,20213.000000,3.110811e+06,82.000000,124.000000,137.000000,125.000000,130.000000,190.000000,78.000000,129.000000,126.00000,115.000000,125.000000,225.000000
75%,20223.000000,3.120133e+06,180.000000,261.000000,286.000000,254.000000,265.000000,386.000000,168.000000,274.000000,266.00000,242.750000,263.000000,459.000000
max,20232.000000,3.130327e+06,1662.000000,2161.000000,2148.000000,2235.000000,1577.000000,2260.000000,1639.000000,2812.000000,2070.00000,2045.000000,1603.000000,2921.000000


#### 중복행 확인 및 제거
- 중복행 없음

In [22]:
resident_popul[resident_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수


### 직장인구(working_popul) 테이블

#### 불필요한 feature 제거
- 총 상주인구 수
- 남성 상주인구 수
- 여성 상주인구 수
- 연령대 10 ~ 60 이상상주인구 수

In [19]:
working_popul.drop(['총_직장_인구_수','남성_직장_인구_수','여성_직장_인구_수','연령대_10_직장_인구_수','연령대_20_직장_인구_수','연령대_30_직장_인구_수','연령대_40_직장_인구_수','연령대_50_직장_인구_수','연령대_60_이상_직장_인구_수'],axis=1,inplace=True)

#### 결측치 확인 및 제거
- 결측치 없음

In [20]:
working_popul.isnull().sum()

기준_년분기_코드              0
상권_구분_코드               0
상권_구분_코드_명             0
상권_코드                  0
상권_코드_명                0
남성연령대_10_직장_인구_수       0
남성연령대_20_직장_인구_수       0
남성연령대_30_직장_인구_수       0
남성연령대_40_직장_인구_수       0
남성연령대_50_직장_인구_수       0
남성연령대_60_이상_직장_인구_수    0
여성연령대_10_직장_인구_수       0
여성연령대_20_직장_인구_수       0
여성연령대_30_직장_인구_수       0
여성연령대_40_직장_인구_수       0
여성연령대_50_직장_인구_수       0
여성연령대_60_이상_직장_인구_수    0
dtype: int64

#### 이상치 확인 및 제거

In [21]:
working_popul.describe()

,기준_년분기_코드,상권_코드,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
count,25989.000000,2.598900e+04,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000,25989.000000
mean,20212.357728,3.115049e+06,3.362153,150.044173,296.655893,283.567432,205.258648,121.233060,4.495402,194.862519,209.883989,160.092462,122.036362,59.625842
std,12.108494,1.026138e+04,19.451383,603.797875,1199.913144,1096.211066,699.620773,308.133216,25.992142,853.121595,890.727983,586.563498,434.896462,178.393298
min,20191.000000,3.001491e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20202.000000,3.110403e+06,0.000000,4.000000,10.000000,14.000000,13.000000,10.000000,0.000000,5.000000,8.000000,11.000000,10.000000,4.000000
50%,20212.000000,3.110810e+06,0.000000,14.000000,31.000000,42.000000,41.000000,32.000000,0.000000,20.000000,25.000000,32.000000,30.000000,14.000000
75%,20222.000000,3.120128e+06,1.000000,55.000000,117.000000,129.000000,118.000000,97.000000,1.000000,64.000000,80.000000,82.000000,71.000000,40.000000
max,20232.000000,3.130327e+06,604.000000,14967.000000,28790.000000,24162.000000,16294.000000,6577.000000,762.000000,29698.000000,32760.000000,19197.000000,10550.000000,3412.000000


#### 중복행 확인 및 제거
- 중복행 없음

In [24]:
working_popul[working_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수


### 임대료 데이터

In [27]:
small_rent

,상권별(1),상권별(2),상권별(3),2023.2/4,2023.3/4
0,전국,소계,소계,19.4,19.4
1,서울,소계,소계,49.2,49.5
2,서울,소계(도심지역),소계,66.4,66.8
3,서울,소계(도심지역),광화문,82.5,83.0
4,서울,소계(도심지역),남대문,63.9,64.0
...,...,...,...,...,...
249,제주,소계,소계,14.3,14.5
250,제주,광양사거리,소계,12.3,12.3
251,제주,노형오거리,소계,20.1,20.2
252,제주,서귀포도심,소계,17.0,17.4


In [32]:
area.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872


In [33]:
rent.head()

,2022,4,11650660,내곡동,01,1층,3286492,124345,157210,13
0,2022,4,11650660,내곡동,2,1층외,17778757,110169,287957,8
1,2022,4,11650660,내곡동,0,전체층,8807355,118945,207018,21
2,2022,4,11650620,방배3동,2,1층외,3590641,92720,128626,12
3,2022,4,11650620,방배3동,0,전체층,2245880,93565,116024,26
4,2022,4,11650620,방배3동,1,1층,1093228,94289,105221,14
